# LangChain for LLM Application Development
when building a LLM application, there are often reusable models. We repeatedly prompt a model, parses output. Langchain gices an easy set of abstraction to do these operation. 
- open source development framework for LLM applications
- Python and Javascript (TypeScript) packages
- Focused on compostion and modularity
- Key Components- Use cases: common ways to combine componets

### Models
- LLMs
- Chat Models
- Test Embedding Models

### Prompts
how  you get models to do useful things
- prompt Templates
- Output parsers
    - Retry/ fixing
- Example selection

### Indexes
way to ingesting data  so that we can combine it with model
- document Loaders
- test splitters
- vector stores
- retrievers

### Chains
end to end use cases
- Prompt + LLM + Output Parsing
- Can be used as building blocks for lnger chains
- More application specific chains

### Agents
- Agent Types
    - Algorithm for getting LLMs to use tool
- Agent Toolkits
    - Agents armed with specific tools for a specific applications

---
    
# LangChain: Models, Prompts and Output Parsers
* Models refers to langague models
* Prompts refers to style of creating inputs to pass into the models
* Output parsers are oppositie to prompts, its taking output of the models and parsing it into more structuresd output. Parser parses the outp to store data in python dictionary or other format. 

## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get the [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [1]:
# Install necessary openAI packages.

#!pip install python-dotenv
#!pip install openai

In [2]:
# import necessary libraries and laod openAI secret key
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [4]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"


print('Available LLM model is ',llm_model)

In [7]:
print('Available LLM model is ',llm_model)

Available LLM model is  gpt-3.5-turbo


## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

To motivate the LangChain abstractions for models, prompts and parsers. Lets say you get an email in other language than english and ask the LLM to translate the text to American english in calm and respctfult tone

In [9]:
# helper function
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]


Call the helper funciton. This will call call the chatGPt with the type of model we described in helper funciton 
(llm_model = "gpt-3.5-turbo") to given an answer

In [30]:
get_completion("What is 1+1?")
# As an AI langague model, I can tell you that the answer to 1+1=2

'1+1 equals 2.'

In [11]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [26]:
# choose your style to translate text.
style = """French \
in a calm and respectful tone
"""

## translate the text in calm and respsctful tone.
# style = """American English \
# in a calm and respectful tone
# """

In [27]:
# To do this specify a prompt using a `f` string

prompt = f"""I have Translate the customer_email \
that is delimited by triple backticks 
into a style that is mentioned by me {style}.
text: ```{customer_email}```
"""

print(prompt) # this generates a prompt

I have Translate the customer_email that is delimited by triple backticks 
into a style that is mentioned by me French in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [31]:
# prompt the large language model to get a response
response = get_completion(prompt)
response

"Bonjour cher client,\n\nJe suis désolé d'apprendre que le couvercle de votre mixeur s'est envolé et a éclaboussé les murs de votre cuisine avec du smoothie! De plus, il est regrettable que la garantie ne couvre pas les frais de nettoyage de votre cuisine. Je suis là pour vous aider immédiatement, mon ami!\n\nCordialement,"

If ou have differnt customers writing differnt review in differnt languages like french,  German etc.  You can imagine that having generate a whole sequence of prompts to generate such translations. How can we do that in more conveinient way using Langchain

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [ ]:
#!pip install --upgrade langchain

### Model

In [32]:
# import ChatOpenAI, this is langcahin abstraction for chatGPT API endpoint
from langchain.chat_models import ChatOpenAI

In [33]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat
# creates an object, whcih uses chatGPT model 'gpt-3.5-turbo'

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

When I'm building applications, one thisg that we often do is to set `temperature=0.0` however default parameter is 0.7 to make output little less random.

### Prompt template

In [34]:
# define tamplate string
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [35]:
# to repetedly use tha bove template, import langcahins ChatPromptTemplate
from langchain.prompts import ChatPromptTemplate

# create prompt template using prompt template_string that we wrote above
prompt_template = ChatPromptTemplate.from_template(template_string)


In [36]:
# from the prompt template we can extract the original prompt. We can 
# see that this tempalte has two input variables ['style', 'text']. We 
# can see the original template that we have specified also

prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)

In [37]:
# check the input varaible si prompt template
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [39]:
# Specify the style
customer_style = """American English \
in a calm and respectful tone
"""

In [40]:
# customer Email/message
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [41]:
# create customer mesages to generate prompt, using stype and text
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [42]:
# check the type of the messages
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.HumanMessage'>


In [44]:
# first element of the list, whcih is basically the prompt we are creating
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} example=False


In [45]:
# Call the LLM to translate to the style of the customer message.
# call `chat` which we defined earlier for openAI chat reference. 
#  chat = ChatOpenAI(temperature=0.0, model=llm_model)
customer_response = chat(customer_messages)

In [46]:
# pritn the customer response content. translate from english pirate to Amercian engligh
print(customer_response.content)

Oh man, I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And on top of that, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, buddy!


In [47]:
# lets say customer replied in the origianl language, englsijh speaking 
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [48]:
# translate in to pirate tone
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [49]:
# prompt
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [50]:
# chat GPT response
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, valued customer! Regrettably, the warranty be not coverin' the cost o' cleanin' yer galley due to yer own negligence. Ye see, 'twas yer own doin' when ye forgot to secure the lid afore startin' the blender. 'Tis a tough break, indeed! Fare thee well, matey!


## Why are we using prompt template instead of f string.

- The answer is when we build sophisticated applications the prompt can be long and detailed. 
- prompt templates are usefule abstractions to re use them when required. 
- prompt format it in a particual r format and wrapping in a particualr format to be used by gpt.
- Langchain also provides prompts for common operations.
- Langchain supports output parsing. Langchain library fucntions parse the LLM's output assuming output that it will use certain keywords. Example Thought, Actions, Observations as keywords for chain-of-thoughts reasonign (framwork called reAct) building complex LLM, we instruct LLM to genrate output in a certain format.

## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [51]:
# extract information from product a review and format that output in Json format.
# here we define the format as python dictionary
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [52]:
# example of customer review and format of output.
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

# review template as the LLM to take the customer review and give output as the given keys
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [53]:
from langchain.prompts import ChatPromptTemplate

# create prompt tempalte from the review tempalte
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [54]:
# create the mesage
messages = prompt_template.format_messages(text=customer_review)
# creat teh openAI endpoint
chat = ChatOpenAI(temperature=0.0, model=llm_model)

# call the openAI endpoint
response = chat(messages)
print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": "It's slightly more expensive than the other leaf blowers out there"
}


In [55]:
# check the type of the response. Its a string not json or dictionary  
type(response.content)

str

In [56]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [57]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [58]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")

price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

# wrap all there in a list
response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [59]:

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [60]:
# now langchain can give you the prompt by itself
format_instructions = output_parser.get_format_instructions()

In [61]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [62]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [63]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "

In [64]:
response = chat(messages)

In [65]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [66]:
output_dict = output_parser.parse(response.content)

In [67]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [68]:
type(output_dict)

dict

In [69]:
output_dict.get('delivery_days')

2